## Import Libraries

In [1]:
import whisper
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm

## Load Whisper Model

Using the `base` model for a good balance between speed and accuracy. You can change to `tiny`, `small`, `medium`, or `large` based on your needs.

In [2]:
import subprocess
import shutil

# Check if ffmpeg is available
ffmpeg_path = shutil.which("ffmpeg")

if ffmpeg_path:
    print(f"✅ FFmpeg found at: {ffmpeg_path}")
    # Check version
    try:
        result = subprocess.run(["ffmpeg", "-version"], capture_output=True, text=True)
        version_line = result.stdout.split('\n')[0]
        print(f"   {version_line}")
    except:
        print("   (Could not check version)")
else:
    print("❌ FFmpeg NOT found!")
    print("\n⚠️  Please install ffmpeg:")
    print("   Option 1: winget install \"FFmpeg (Essentials Build)\"")
    print("   Option 2: Download from https://ffmpeg.org/download.html")
    print("   Option 3: Install via Chocolatey: choco install ffmpeg")
    print("\n   After installation, restart this notebook kernel.")

✅ FFmpeg found at: C:\Users\tomer\AppData\Local\Microsoft\WinGet\Packages\Gyan.FFmpeg.Essentials_Microsoft.Winget.Source_8wekyb3d8bbwe\ffmpeg-8.0.1-essentials_build/bin\ffmpeg.EXE
   ffmpeg version 8.0.1-essentials_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers


## Check FFmpeg Installation

Whisper requires ffmpeg to process audio files. Let's verify it's installed.

In [3]:
print("Loading Whisper model...")
model = whisper.load_model("base")
print("Model loaded successfully!")

Loading Whisper model...
Model loaded successfully!


## Transcribe Audio Files

Process all audio files from `good_answers` and `bad_answers` folders, then save transcripts to separate output folders.

In [4]:
def transcribe_folder(input_folder, output_folder, prefix):
    """
    Transcribe all audio files in input_folder and save as text files in output_folder.
    
    Args:
        input_folder: Path to folder containing audio files
        output_folder: Path to save transcript text files
        prefix: Prefix for output files (e.g., 'good' or 'bad')
    """
    input_path = Path(input_folder)
    output_path = Path(output_folder)
    
    # Create output directory if it doesn't exist
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Get all audio files (common audio extensions)
    audio_extensions = {'.mp3', '.wav', '.m4a', '.flac', '.ogg', '.opus', '.webm'}
    audio_files = sorted([f for f in input_path.iterdir() 
                         if f.is_file() and f.suffix.lower() in audio_extensions])
    
    if not audio_files:
        print(f"⚠️  No audio files found in {input_folder}")
        return
    
    print(f"\n📁 Processing {len(audio_files)} files from {input_folder}...")
    
    results = []
    
    for idx, audio_file in enumerate(tqdm(audio_files, desc=f"Transcribing {prefix}")):
        try:
            # Transcribe audio
            result = model.transcribe(str(audio_file), language="he")  # Change language as needed
            transcript = result["text"]
            
            # Save transcript
            output_file = output_path / f"{prefix}_{idx+1:03d}.txt"
            output_file.write_text(transcript, encoding='utf-8')
            
            results.append({
                'original_file': audio_file.name,
                'transcript_file': output_file.name,
                'text_length': len(transcript)
            })
            
        except Exception as e:
            print(f"❌ Error processing {audio_file.name}: {e}")
            results.append({
                'original_file': audio_file.name,
                'transcript_file': 'ERROR',
                'text_length': 0
            })
    
    print(f"✅ Completed {prefix} transcriptions!")
    return pd.DataFrame(results)

### Process Good Answers

In [5]:
good_df = transcribe_folder(
    input_folder="../data/good_answers",
    output_folder="../data/good_answer_transcripts",
    prefix="good"
)

# Display summary
if good_df is not None:
    display(good_df)


📁 Processing 33 files from ../data/good_answers...


Transcribing good:   0%|          | 0/33 [00:00<?, ?it/s]c:\Users\tomer\OneDrive\מסמכים\GitHub\customer-support-llm\.venv\lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Transcribing good:   6%|▌         | 2/33 [00:58<15:00, 29.05s/it]


KeyboardInterrupt: 

### Process Bad Answers

In [ ]:
bad_df = transcribe_folder(
    input_folder="../data/bad_answers",
    output_folder="../data/bad_answer_transcripts",
    prefix="bad"
)

# Display summary
if bad_df is not None:
    display(bad_df)


📁 Processing 13 files from ../data/bad_answers...



📁 Processing 13 files from ../data/bad_answers...


Transcribing bad:   0%|          | 0/13 [00:00<?, ?it/s]


📁 Processing 13 files from ../data/bad_answers...


Transcribing bad:   0%|          | 0/13 [00:00<?, ?it/s]

...

c:\Users\tomer\OneDrive\מסמכים\GitHub\customer-support-llm\.venv\lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Transcribing bad:  23%|██▎       | 3/13 [00:00<00:00, 29.81it/s]

❌ Error processing bad_003.m4a: [WinError 2] The system cannot find the file specified
❌ Error processing bad_004.m4a: [WinError 2] The system cannot find the file specified
❌ Error processing bad_005.m4a: [WinError 2] The system cannot find the file specified
❌ Error processing bad_006.m4a: [WinError 2] The system cannot find the file specified
❌ Error processing bad_007.m4a: [WinError 2] The system cannot find the file specified
❌ Error processing bad_008.m4a: [WinError 2] The system cannot find the file specified
❌ Error processing bad_009.m4a: [WinError 2] The system cannot find the file specified
❌ Error processing bad_010.m4a: [WinError 2] The system cannot find the file specified


Transcribing bad: 100%|██████████| 13/13 [00:00<00:00, 49.85it/s]

❌ Error processing bad_011.m4a: [WinError 2] The system cannot find the file specified
❌ Error processing bad_012.m4a: [WinError 2] The system cannot find the file specified
❌ Error processing bad_013.m4a: [WinError 2] The system cannot find the file specified
✅ Completed bad transcriptions!


,original_file,transcript_file,text_length
0,bad_001.m4a,ERROR,0
1,bad_002.m4a,ERROR,0
2,bad_003.m4a,ERROR,0
3,bad_004.m4a,ERROR,0
4,bad_005.m4a,ERROR,0
5,bad_006.m4a,ERROR,0
6,bad_007.m4a,ERROR,0
7,bad_008.m4a,ERROR,0
8,bad_009.m4a,ERROR,0
9,bad_010.m4a,ERROR,0


## Summary Statistics

In [ ]:
print("=" * 50)
print("TRANSCRIPTION SUMMARY")
print("=" * 50)

if good_df is not None:
    print(f"\n✅ Good Answers:")
    print(f"   Files processed: {len(good_df)}")
    print(f"   Avg text length: {good_df['text_length'].mean():.0f} chars")
    print(f"   Total text: {good_df['text_length'].sum():,} chars")

if bad_df is not None:
    print(f"\n❌ Bad Answers:")
    print(f"   Files processed: {len(bad_df)}")
    print(f"   Avg text length: {bad_df['text_length'].mean():.0f} chars")
    print(f"   Total text: {bad_df['text_length'].sum():,} chars")

print("\n" + "=" * 50)

TRANSCRIPTION SUMMARY

✅ Good Answers:
   Files processed: 33
   Avg text length: 0 chars
   Total text: 0 chars

❌ Bad Answers:
   Files processed: 13
   Avg text length: 0 chars
   Total text: 0 chars

